In [1]:
import argparse
import os
from functools import partial
import numpy as np
import torch
from monai.inferers import sliding_window_inference
from monai.losses import DiceLoss
from monai.metrics import DiceMetric
from monai.networks.nets import SwinUNETR
from monai.transforms import Activations, AsDiscrete, Compose
from monai.utils.enums import MetricReduction

2025-12-23 01:41:56.284135: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-23 01:41:56.330642: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-23 01:41:57.114529: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
import os
print(os.getcwd())


/home/ai2lab/workshop_February


In [3]:
import argparse
import sys

def get_args():
    parser = argparse.ArgumentParser(description="Swin UNETR for Automated Brain Tumor Segmentation")

    parser.add_argument("--data_dir", default="/home/ai2lab/extension_to_brats_challenge/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData", type=str, help="Dataset directory")
    parser.add_argument("--output_dir",default="/home/ai2lab/workshop_February/dataset",type=str, help="output directory")
    parser.add_argument("--save_checkpoint", action="store_true", help="Save checkpoint during training")
    parser.add_argument("--max_epochs", default=200, type=int, help="Max number of training epochs")
    parser.add_argument("--batch_size", default=1, type=int, help="Batch size")

    # Detect if running in Jupyter
    if "ipykernel" in sys.modules:
        # Jupyter: ignore sys.argv to prevent conflicts
        return parser.parse_args(args=[])
    else:
        # Standard script: parse normally
        return parser.parse_args()

# Get arguments
args = get_args()

# Example usage
print(args)


Namespace(data_dir='/home/ai2lab/extension_to_brats_challenge/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData', output_dir='/home/ai2lab/workshop_February/dataset', save_checkpoint=False, max_epochs=200, batch_size=1)


In [4]:
from monai.transforms import MapTransform
from monai.transforms import (
     ToTensord,
    LoadImaged,
RandSpatialCropd,
NormalizeIntensityd
)

class ConvertToMultiChannelBasedOnCustomBratsClassesd(MapTransform):
    """
    Converts label values to multi-channel format for BraTS-like task.
    Your dataset label IDs:
    - 1: necrosis/NCR
    - 2: edema
    - 3: enhancing tumor (ET)

    Channels:
    - Channel 0: Tumor Core (TC) = 1 + 3
    - Channel 1: Whole Tumor (WT) = 1 + 2 + 3
    - Channel 2: Enhancing Tumor (ET) = 3
    """

    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            seg = d[key]  # (C, H, W, D) or (H, W, D)
            
            if isinstance(seg, torch.Tensor):
                seg = seg.numpy()
            
           
            
            
            # make sure we're working with 3D (no extra channel dim)
            if seg.ndim == 4 and seg.shape[0] == 1:
                seg = np.squeeze(seg, axis=0)
            
            seg = np.where(seg == 4, 3, seg)
            tc = np.logical_or(seg == 1, seg == 3)   # Tumor Core
            wt = np.logical_or(tc, seg == 2)         # Whole Tumor
            et = seg == 3                             # Enhancing Tumor

            multi_channel = np.stack([tc, wt, et], axis=0).astype(np.float32)  # (3, H, W, D)
            d[key] = multi_channel
        return d
# For training (includes segmentation if available)
def print_shape(d):
    for k, v in d.items():
        print(f"{k}: {v.shape}")
    return d

    
class LoadNumpyd(MapTransform):
    def __init__(self, keys, allow_missing_keys=False):
        super().__init__(keys)
        self.allow_missing_keys = allow_missing_keys

    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            if key not in d:
                if self.allow_missing_keys:
                    continue
                else:
                    raise KeyError(f"Key '{key}' not found in data and allow_missing_keys=False")

            arr = np.load(d[key])  # (1, 128, 768)
            arr = np.squeeze(arr, axis=0)  # (128, 768)
            arr = arr.astype(np.float32)

            d[key] = arr
        return d



train_transforms = Compose([
    LoadImaged(keys=["img",  "seg"], allow_missing_keys=True, ensure_channel_first=True),
    LoadNumpyd(keys=["text_feature"],allow_missing_keys=True),
    ConvertToMultiChannelBasedOnCustomBratsClassesd(keys="seg", allow_missing_keys=True),
    RandSpatialCropd(keys=["img", "seg"], roi_size=(128,128,128), random_center=True, random_size=False, allow_missing_keys=True),
    NormalizeIntensityd(keys="img", nonzero=True, channel_wise=True),
    ToTensord(keys=["img", "seg","text_feature"], dtype=torch.float32, allow_missing_keys=True),    
])    

val_transforms = Compose([
    LoadImaged(keys=["img",  "seg"], ensure_channel_first=True,allow_missing_keys=True),
    LoadNumpyd(keys=["text_feature"],allow_missing_keys=True),
    ConvertToMultiChannelBasedOnCustomBratsClassesd(keys="seg",allow_missing_keys=True),
    NormalizeIntensityd(keys="img", nonzero=True, channel_wise=True),
    ToTensord(keys=["img", "seg","text_feature"],dtype=torch.float32, allow_missing_keys=True),
])

In [5]:
##########preparing dataset############
# from pathlib import Path
# import shutil
# # Modify these paths
# data_root = Path(args.data_dir)  # contains BraTS-GoAT-00000, BraTS-GoAT-00001, etc.
# output_root = Path(args.output_dir)
# imagesTr = output_root / "imagesTr"
# labelsTr = output_root / "labelsTr"

# # Create directories
# imagesTr.mkdir(parents=True, exist_ok=True)
# labelsTr.mkdir(parents=True, exist_ok=True)

# # Mapping from suffix to nnUNet modality index
# modality_map = {
#     "t1": 0,
#     "t1ce": 1,
#     "t2": 2,
#     "flair": 3
# }
# for case_dir in data_root.iterdir():
#     if not case_dir.is_dir():
#         continue
#     case_id = case_dir.name  # e.g., BraTS20_Training_001
#     print(case_id)
#     for mod, idx in modality_map.items():
#         src_file = case_dir / f"{case_id}_{mod}.nii"
#         dst_file = imagesTr / f"{case_id}_{idx:04d}.nii"
#         if src_file.exists():
#             shutil.copy(src_file, dst_file)
#                 # print('no need')
#         else:
#             print(f"Missing modality file: {src_file}")

#         # Handle label
#     label_src = case_dir / f"{case_id}_seg.nii"
#     label_dst = labelsTr / f"{case_id}.nii"
#     if label_src.exists():
#         shutil.copy(label_src, label_dst)
#     else:
#         print(f"Warning: No label found for {case_id}, creating dummy")
            

print("✅ Data copying complete.")


✅ Data copying complete.


In [6]:
def load_data(directory_name, output_dir):
    """
    Load the dataset's training and validation splits from CSV files
    """

    path = directory_name
    label_path=os.path.join(path,'labelsTr')
    print(label_path)
    text_feature_root=os.path.join(path,'text_data/TextBraTSData')
    
    print(text_feature_root)
    
    val_csv_path = os.path.join(path, "imagesTr/validation_set.csv")
    train_csv_path = os.path.join(path, "imagesTr/train_set.csv")
    
    print('check path:',val_csv_path,train_csv_path)

    if os.path.exists(val_csv_path) and os.path.exists(train_csv_path):
        print("Loading existing splits...")
        
        val_data = pd.read_csv(val_csv_path)
        train_data = pd.read_csv(train_csv_path)
    else:
        print("If you want to perform new split...")
        all_files = sorted([f for f in os.listdir(path) if f.endswith('.nii.gz')])
        subject_ids = sorted(set(re.sub(r'_\d{4}\.nii\.gz$', '', f) for f in all_files))
        df = pd.DataFrame({'SubjectID': subject_ids})

        train_data, val_data = train_test_split(df, test_size=0.20, random_state=42)

        train_data['split'] = 'train'
        val_data['split'] = 'val'
    

        final_df = pd.concat([train_data, val_data]).sort_values(by='SubjectID')
        output_csv = os.path.join(output_dir, 'train_val_dataset_split.csv')
        final_df.to_csv(output_csv, index=False)

     
        # Save each split separately
        train_data.to_csv(train_csv_path, index=False)
        val_data.to_csv(val_csv_path, index=False)
        

    # Build final dataset structure
    def get_modality_paths(subject_id):
        path1=os.path.join(path,"imagesTr")
        return [
            os.path.join(path1, f"{subject_id}_0000.nii"),
            os.path.join(path1, f"{subject_id}_0001.nii"),
            os.path.join(path1, f"{subject_id}_0002.nii"),
            os.path.join(path1, f"{subject_id}_0003.nii")
        ]

    
   
    def build_data_list(df_split):
        data_list = []
        for sid in df_split['SubjectID']:
            img_paths = get_modality_paths(sid)
            seg_path = os.path.join(label_path, f"{sid}.nii")
            text_feature_file = os.path.join(text_feature_root, sid, f"{sid}_flair_text.npy")
            item = {
                "img": img_paths,
                "subject_id": sid,
                "seg": seg_path,
                "text_feature": text_feature_file            
            }
    
            data_list.append(item)
        return data_list
    
    
    
    filenames_train = build_data_list(train_data)
    filenames_val = build_data_list(val_data)
    ds_train = Dataset(data=filenames_train, transform=train_transforms)
    ds_val = Dataset(data=filenames_val, transform=val_transforms)
    train_loader = DataLoader(ds_train, num_workers=4, batch_size=4, shuffle=True, pin_memory=True, drop_last=True)
    val_loader = DataLoader(ds_val, num_workers=4, batch_size=2, shuffle=False, pin_memory=True, drop_last=False)
    
    # Example: print first batch from train_loader
    # for batch in train_loader:
    #     # Check keys in the batch
    #     print("Batch keys:", batch.keys())
    
    #     # Example: image data
    #     images = batch["img"]  
    #     print("images shape:", images.shape)
    #     print("images dtype:", images.dtype)
    #     print("images device:", images.device if hasattr(images, "device") else "N/A")
    #     print("images min/max/mean:", images.min().item(), images.max().item(), images.mean().item())
    
    #     # Example: text features
    #     if "text_feature" in batch:
    #         text_feat = batch["text_feature"]
    #         print("text_feat shape:", text_feat.shape)
    #         print("text_feat dtype:", text_feat.dtype)
    #         print("text_feat min/max/mean:", text_feat.min().item(), text_feat.max().item(), text_feat.mean().item())
    
    #     # Only inspect the first batch
    #     break


    return train_loader, val_loader





In [7]:
 # === Step 1: Load data ===
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import torch
import re
from monai.data import Dataset
from torch.utils.data import DataLoader

directory_name=args.output_dir
output_dir=args.output_dir
train_loader, val_loader= load_data(directory_name,output_dir)
for batch in train_loader:
        print(batch["text_feature"].shape, batch["img"].shape, batch["seg"].shape)
        break


/home/ai2lab/workshop_February/dataset/labelsTr
/home/ai2lab/workshop_February/dataset/text_data/TextBraTSData
check path: /home/ai2lab/workshop_February/dataset/imagesTr/validation_set.csv /home/ai2lab/workshop_February/dataset/imagesTr/train_set.csv
Loading existing splits...
torch.Size([4, 128, 768]) torch.Size([4, 4, 128, 128, 128]) torch.Size([4, 3, 128, 128, 128])


In [12]:
import pandas as pd
import numpy as np
import nibabel as nib
import re
from monai.data import decollate_batch
import nibabel as nib
from monai.transforms import AsDiscrete, Compose, EnsureType, Activations
from monai.metrics import DiceMetric
from monai.utils.enums import MetricReduction
from functools import partial
from monai.inferers import sliding_window_inference
from functools import partial
from monai.losses import DiceLoss
import torch.nn.functional as F
from monai.losses import FocalLoss

def convert_to_single_channel(multi_channel_np: np.ndarray) -> np.ndarray:
    """
    Convert BraTS-style one-hot (3, H, W, D) prediction or GT to single-channel label map:
        0: Background
        1: Tumor Core (TC) [label 1 in GT]
        2: Edema [label 2 in GT]
        3: Enhancing Tumor (ET) [label 3 in GT]

    Assumes:
        Channel 0: TC = 1 + 3
        Channel 1: WT = 1 + 2 + 3
        Channel 2: ET = 3
    """
    assert multi_channel_np.shape[0] == 3, "Expected 3 channels (TC, WT, ET)"
    
    tc = multi_channel_np[0]
    et = multi_channel_np[2]

    output = np.zeros_like(tc, dtype=np.uint8)

    # Priority-based assignment
    output[tc == 1] = 1  # TC gets label 1 (includes necrosis and ET)
    output[(multi_channel_np[1] == 1) & (tc == 0) & (et == 0)] = 2  # Edema only gets label 2
    output[et == 1] = 3  # Enhancing Tumor gets label 3 (overwrites TC if needed)

    return output


def train(train_loader, val_loader, model, optimizer, scheduler, max_epochs, directory_name, start_epoch=1):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)            # Move model to cuda:0
    model.train()
    results_dir=os.path.join(directory_name,"results")
    os.makedirs(results_dir,exist_ok=True)

    criterion = DiceLoss(to_onehot_y=False, sigmoid=True)
    criterion_ce = nn.BCEWithLogitsLoss()
    
    post_sigmoid = Activations(sigmoid=True)
    post_pred = AsDiscrete(argmax=False, threshold=0.5)
    dice_metric = DiceMetric(include_background=True, reduction=MetricReduction.MEAN_BATCH, get_not_nans=True)

    checkpoint_path = os.path.join(directory_name, "best_model.pth")
    last_model_path = os.path.join(directory_name, "last_model.pth")
    training_results_dir = os.path.join(directory_name, "training_results")
    os.makedirs(training_results_dir, exist_ok=True)

    best_val_loss = float("inf")
    best_dice_score=-1.0
    if os.path.exists(last_model_path):
        checkpoint = torch.load(last_model_path, map_location=device)
     
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        scheduler.load_state_dict(checkpoint['scheduler'])
        # best_val_loss = checkpoint.get('best_val_loss', float("inf"))
        best_dice_score=checkpoint.get('best_dice_score',-1)
        start_epoch = checkpoint.get('epoch', 1) + 1
        print(f"Last model loaded. Resuming training from epoch: {start_epoch}")
        print(f"Resuming with best Dice score: {best_dice_score:.4f}")



    for epoch in range(start_epoch, max_epochs + 1):
        print(f"\n🔁 Epoch {epoch}")
        model.train()
        train_loss = 0.0

        for batch in train_loader:
            img = batch["img"].to(device)
            seg = batch.get("seg").to(device)
            text=batch.get("text_feature").to(device)
            B, C, H, W, D = img.shape
            
            optimizer.zero_grad()
            pred_seg = model(img,text)
          
            loss_seg = criterion(pred_seg, seg) + criterion_ce(pred_seg, seg)

            train_loss += loss_seg.item()
            loss_seg.backward()
            optimizer.step()
            

        train_loss /= len(train_loader)
        print(f"✅ Training Loss: {train_loss:.4f}")

        # ----------------------
        # Validation
        # ----------------------
        
        model.eval()
        val_loss = 0.0
        dice_scores = []
        import numpy as np
    
        affine = np.eye(4)

        with torch.no_grad():
            dice_metric.reset()
            for batch_idx, batch in enumerate(val_loader):
                    img = batch["img"].to(device)
                    seg = batch.get("seg").to(device)
                    text = batch.get("text_feature").to(device)
    
        
                    # Define a predictor lambda that includes text_feature
                    predictor_with_text = lambda x: model(x, text)
                    
                    # Create a sliding_window_inference instance with this predictor
                    model_inferer_with_text = partial(
                        sliding_window_inference,
                        roi_size=[128,128,128],
                        sw_batch_size=2,
                        predictor=predictor_with_text,
                        overlap=0.7,
                    )
                    
                    # Run inference
                    pred_seg = model_inferer_with_text(img)
                    # val_output_convert = [post_pred(post_sigmoid(p)) for p in pred_seg]
                    # pred_seg = [p for p in zip(val_output_convert)]
                    # true_seg = [s for s in zip(seg)]
                    pred = post_pred(post_sigmoid(pred_seg))


                        
                    dice_metric(y_pred=pred, y=seg)
                    for i in range(img.shape[0]):
                        subject_id = batch["subject_id"][i]

            
                        img_paths = [os.path.join(directory_name, "imagesTr", f"{subject_id}_0000.nii.gz")]
    
                        img_path = img_paths[0]
                        save_filename = f"{subject_id}"
                                
                        save_img_path = os.path.join(results_dir, f"{save_filename}_gt.nii.gz")
                        save_pred_path = os.path.join(results_dir, f"{save_filename}_pred.nii.gz")
                                
                        affine = nib.load(img_path).affine
                        pred_np = pred[i].detach().cpu().numpy().astype(np.uint8)
                        seg_np = seg[i].detach().cpu().numpy().astype(np.uint8)            
                        single_channel_pred = convert_to_single_channel(pred_np)
                        single_channel_gt = convert_to_single_channel(seg_np)
                                   # Save this single-channel prediction as NIfTI
                        nib.save(nib.Nifti1Image(single_channel_pred, affine), save_pred_path)
                        nib.save(nib.Nifti1Image(single_channel_gt, affine), save_img_path)
              
        
        per_class_dice, _ = dice_metric.aggregate()
        mean_dice = per_class_dice.mean().item()
        print(f"Dice Scores — TC: {per_class_dice[0].item():.4f}, "
                  f"WT: {per_class_dice[1].item():.4f}, ET: {per_class_dice[2].item():.4f}")
        print(f"Mean Dice: {mean_dice:.4f}")
        
        if mean_dice > best_dice_score:
                best_dice_score = mean_dice
                torch.save({
                    'epoch': epoch,
                    "state_dict": model.state_dict(),
                    "optimizer": optimizer.state_dict(),
                    "scheduler": scheduler.state_dict(),
                    # "val_loss": val_loss,
                    "best_dice_score": best_dice_score
                }, checkpoint_path)
                print("Best model saved based on Dice score.")
            

    
     
        torch.save({
            'epoch': epoch,
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
            "scheduler": scheduler.state_dict(),
            # "val_loss": val_loss,
            # "best_val_loss": best_val_loss,
            "best_dice_score":best_dice_score
        }, last_model_path)

        # Step the scheduler
        scheduler.step()
    print("Training complete.")




In [13]:
import torch
from monai.networks.nets import SwinUNETR  # or your custom SwinUNet
from monai.optimizers import generate_param_groups
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
import torch.distributed as dist
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
from monai.networks.nets import SwinUNETR


class MultiTaskSwinUNETR_image_text_fusion(nn.Module):
    def __init__(self,
                 # img_size=(256, 256, 160),
                 in_channels=4,
                 feature_size=48,
                 seg_out_channels=3,
                 recon_out_channels=4,
                 use_checkpoint=True,
                 text_embed_dim=768):
        super().__init__()

        self.backbone = SwinUNETR(
            # img_size=img_size,
            in_channels=in_channels,
            out_channels=feature_size,
            feature_size=feature_size,
            use_checkpoint=use_checkpoint)

        # project BioBERT 768 → Swin feature dim (48)
        self.text_proj = nn.Linear(text_embed_dim, feature_size)

        # fuse image+text channels: (2C → C)
        self.fusion = nn.Conv3d(feature_size * 2, feature_size, kernel_size=1)

        # output head
        self.seg_head = nn.Conv3d(feature_size, seg_out_channels, kernel_size=1)
        

    def forward(self, x, text_feature):
    
        # --- Image Backbone ---
        img_features = self.backbone(x)  # [B, C, D, H, W]
        print(f"[Backbone] img_features: {img_features.shape}")

        # --- Text Processing ---
        text_feature = text_feature.mean(dim=1)
        print('check shape:',text_feature.shape)
        text_feat = self.text_proj(text_feature)          # [B, 768] → [B, C]
        print(f"[TextProj] text_feat after Linear: {text_feat.shape}")

        text_feat = text_feat.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)
        print(f"[TextProj] text_feat after unsqueeze: {text_feat.shape}")

        text_feat = text_feat.expand_as(img_features)     # [B, C, D, H, W]
        print(f"[TextProj] text_feat after expand_as: {text_feat.shape}")

        fused = torch.cat([img_features, text_feat], dim=1)  # [B, 2C, D, H, W]
        print(f"[Fusion] concatenated: {fused.shape}")

        fused = self.fusion(fused)                        # [B, C, D, H, W]
        print(f"[Fusion] after Conv3d fusion: {fused.shape}")
        
        # --- Head ---
        seg_output = self.seg_head(fused)
        print(f"[SegHead] seg_output: {seg_output.shape}")


        return seg_output


In [10]:
def main():
    # === Step 2: Set up device ===
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # === Step 3: Initialize model ===
    

    model = MultiTaskSwinUNETR_image_text_fusion(
       # img_size=(256, 256, 160),
       in_channels=4,
       seg_out_channels=3,      # tumor classes
       recon_out_channels=4,    # reconstruct all 4 modalities
       feature_size=48
   )

    # print(model) 
       
    # === Step 4: parameter setup ===
    start_epoch=1
    from torch.optim.lr_scheduler import CosineAnnealingLR
    optimizer = torch.optim.Adam(model.parameters(), lr=4e-4, weight_decay=1e-5)
    scheduler = CosineAnnealingLR(optimizer, T_max=50, eta_min=1e-6)
    max_epochs=200
   
    # === Step 5: Call train() ===
    train(
        train_loader=train_loader,
        val_loader=val_loader,
        model=model,
        optimizer=optimizer,
        scheduler=scheduler,
        max_epochs=max_epochs,
        directory_name=directory_name,
        start_epoch=start_epoch,
        
    )



if __name__ == "__main__":
    main()



🔁 Epoch 1
[Backbone] img_features: torch.Size([4, 48, 128, 128, 128])
check shape: torch.Size([4, 768])
[TextProj] text_feat after Linear: torch.Size([4, 48])
[TextProj] text_feat after unsqueeze: torch.Size([4, 48, 1, 1, 1])
[TextProj] text_feat after expand_as: torch.Size([4, 48, 128, 128, 128])
[Fusion] concatenated: torch.Size([4, 96, 128, 128, 128])
[Fusion] after Conv3d fusion: torch.Size([4, 48, 128, 128, 128])
[SegHead] seg_output: torch.Size([4, 3, 128, 128, 128])


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.00 GiB. GPU 0 has a total capacity of 47.51 GiB of which 2.84 GiB is free. Process 71219 has 13.04 GiB memory in use. Including non-PyTorch memory, this process has 30.47 GiB memory in use. Of the allocated memory 28.14 GiB is allocated by PyTorch, and 1.99 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)